In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from IPython.display import display, HTML

# Fetch the last price using yfinance
def fetch_last_price(ticker):
    try:
        data = yf.Ticker(ticker)
        hist = data.history(period="1d")
        last_price = hist['Close'].iloc[-1]
        return last_price
    except Exception as e:
        print(f"Error fetching last price: {e}")
        return None

# Generate SPX data with strikes and theoretical ES strikes
def generate_spx_data(start=3000, end=7000, step=5):
    spx_strikes = np.arange(start, end + 1, step)
    df = pd.DataFrame({
        "SPX Options Strike": spx_strikes
    })
    
    # Change the multiplier to 1.165%
    df["Multiplied by 1.165%"] = df["SPX Options Strike"] * 1.01165
    
    # Apply rounding to the nearest multiple of 5
    df["ES Theoretical Options Strike"] = df["Multiplied by 1.165%"].apply(lambda x: 5 * round(x / 5))
    
    return df

# Fetch SPX current price
spx_ticker = "^GSPC"
spx_current_price = fetch_last_price(spx_ticker)
closest_multiple = 5 * round(spx_current_price / 5)

# Generate data
spx_data = generate_spx_data()

# Ensure that the 'SPX Last Price' column is created before we use it
spx_data["SPX Last Price"] = np.where(spx_data["SPX Options Strike"] == closest_multiple, spx_current_price, np.nan)

# Now replace NaN with an empty string
spx_data["SPX Last Price"] = spx_data["SPX Last Price"].replace(np.nan, '')  # Replace NaN with empty string

# Add a 'Gamma Type' column if it doesn't already exist
spx_data["Gamma Type"] = ''  # New column to store 'Put Gamma 1', 'Call Gamma 2', etc.

# Reorder the columns so that "Gamma Type" is the first column
spx_data = spx_data[["Gamma Type", "SPX Options Strike", "ES Theoretical Options Strike", "SPX Last Price"]]

# Load gamma exposure data from the CSV file
file_path = 'top_bottom_gamma_exposure.csv'
gamma_exposure_data = pd.read_csv(file_path)

# Sort and get the top 10 highest total gamma exposure for calls and bottom 10 lowest for puts
highest_call_gamma = gamma_exposure_data.nlargest(10, 'TotalGamma')['StrikePrice'].values  # Top 10 Call Gamma (positive)
lowest_put_gamma = gamma_exposure_data.nsmallest(10, 'TotalGamma')['StrikePrice'].values   # Bottom 10 Put Gamma (negative)

# Function to update the Gamma Type column based on inputs
def update_gamma_type():
    spx_data["Gamma Type"] = ""  # Reset the column
    
    # Assign the values to the respective SPX options strikes for Put and Call Gammas
    for i in range(10):
        spx_data.loc[spx_data['SPX Options Strike'] == lowest_put_gamma[i], 'Gamma Type'] = f"Put Gamma {i + 1}"
        spx_data.loc[spx_data['SPX Options Strike'] == highest_call_gamma[i], 'Gamma Type'] = f"Call Gamma {i + 1}"

# Function to highlight rows and apply font size to Gamma Type and SPX Last Price
def highlight_rows(row):
    colors = [''] * len(row)
    text_colors = [''] * len(row)
    font_styles = [''] * len(row)

    # Color mapping for top 5 calls (green) and bottom 5 puts (red)
    gamma_colors = {
        'Call Gamma 1': 'darkgreen',
        'Call Gamma 2': 'forestgreen',
        'Call Gamma 3': 'lightgreen',
        'Call Gamma 4': 'mediumseagreen',
        'Call Gamma 5': 'palegreen',
        'Put Gamma 1': 'darkred',
        'Put Gamma 2': 'firebrick',
        'Put Gamma 3': 'darksalmon',
        'Put Gamma 4': 'lightcoral',
        'Put Gamma 5': 'salmon',
        'Call Gamma 6': 'darkgray',
        'Call Gamma 7': 'darkgray',
        'Put Gamma 6': 'darkgray',
        'Put Gamma 7': 'darkgray',
        'Call Gamma 8': 'gray',
        'Call Gamma 9': 'gray',
        'Put Gamma 8': 'gray',
        'Put Gamma 9': 'gray',
        'Call Gamma 10': 'lightgray',
        'Put Gamma 10': 'lightgray',
    }

    # Apply smaller font size to the Gamma Type column
    font_styles[0] = 'font-size: 75%;'
    
    # Highlight SPX Last Price column in yellow if it has a value
    if row['SPX Last Price'] != '':
        colors[-1] = 'background-color: yellow;'

    # Apply colors based on gamma description
    gamma_type = row['Gamma Type']
    if gamma_type in gamma_colors:
        color = gamma_colors[gamma_type]
        colors = [f'background-color: {color};'] * len(row)
        
        # Set text color based on specific background colors
        if color in ['lightgray', 'lightgreen', 'palegreen', 'mediumseagreen']:
            text_colors = ['color: black;'] * len(row)  # Black text for light backgrounds
        elif 'dark' in color or 'firebrick' in color or 'red' in color:
            text_colors = ['color: white;'] * len(row)  # White text for dark backgrounds

    return [f"{bg} {tc} {fs}" for bg, tc, fs in zip(colors, text_colors, font_styles)]

# Function to display the data
def display_data(focus_on_spx=False):
    update_gamma_type()  # Update Gamma Type column before displaying
    
    focus_index = np.abs(spx_data['SPX Options Strike'] - closest_multiple).idxmin()
    
    lower_bound = max(0, focus_index - 100)
    upper_bound = min(len(spx_data), focus_index + 100)
    
    display_df = spx_data.iloc[lower_bound:upper_bound]

    # Apply the row highlight function and hide the row numbers (index)
    styled = display_df.style.apply(highlight_rows, axis=1).hide(axis='index')
    html = styled.to_html().replace(
        '<table',
        '<table style="height: 400px; overflow-y: auto; display: block; border-collapse: collapse;"'
    ).replace(
        '<thead>',
        '<thead style="position: sticky; top: 0; background-color: black; color: white; z-index: 1;">'  # Header style change
    )
    display(HTML(html))

# Display the data without the search boxes
display_data()  # Initial display


Gamma Type,SPX Options Strike,ES Theoretical Options Strike,SPX Last Price
,5310,5370,
,5315,5375,
,5320,5380,
,5325,5385,
,5330,5390,
,5335,5395,
,5340,5400,
,5345,5405,
,5350,5410,
,5355,5415,
